In [ ]:
"""
Script pour capturer des images de pompes - VERSION OPTIMISÉE
Appuyez sur 'u' pour enregistrer en position HAUTE (up)
Appuyez sur 'd' pour enregistrer en position BASSE (down)
Appuyez sur 'q' pour quitter

Cette version combine:
✓ Réchauffage caméra (évite la première image sombre)
✓ Capture instantanée à la demande (u/d)
✓ Feedback en temps réel
✓ Statistiques détaillées
"""

import cv2
import os
import time
from datetime import datetime

# Chemins
UP_DIR = "./dataset/up"
DOWN_DIR = "./dataset/down"

# Créer les répertoires s'ils n'existent pas
os.makedirs(UP_DIR, exist_ok=True)
os.makedirs(DOWN_DIR, exist_ok=True)

def capture_images():
    """Capture des images avec la webcam et les classe en temps réel"""
    
    print(" Initialisation de la webcam...")
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print(" Erreur: Impossible d'ouvrir la webcam")
        return
    
    # Réchauffage caméra (évite première image sombre)
    print(" Réchauffage de la caméra... (2 secondes)")
    for i in range(10):
        cap.read()
        time.sleep(0.2)
    
    # Compter les images existantes
    up_count = len([f for f in os.listdir(UP_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))])
    down_count = len([f for f in os.listdir(DOWN_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))])
    
    session_up = 0
    session_down = 0
    
    print("\n" + "=" * 60)
    print(" CAPTURE DE DATASET - POMPES")
    print("=" * 60)
    print("\n Commandes:")
    print("  'u' - Enregistrer position HAUTE (bras étendus)")
    print("  'd' - Enregistrer position BASSE (bras plié)")
    print("  'q' - Quitter")
    print("\n" + "-" * 60 + "\n")
    
    frame_count = 0
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print(" Erreur: Impossible de lire la webcam")
            break
        
        frame_count += 1
        
        # Redimensionner pour affichage
        frame = cv2.resize(frame, (1000, 720))
        h, w = frame.shape[:2]
        
        # Overlay semi-transparent pour l'info
        overlay = frame.copy()
        cv2.rectangle(overlay, (0, 0), (w, 120), (0, 0, 0), -1)
        cv2.addWeighted(overlay, 0.3, frame, 0.7, 0, frame)
        
        # Affichage des comptes
        cv2.putText(frame, f"TOTAL: {up_count + down_count} images", 
                   (20, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)
        cv2.putText(frame, f"UP: {up_count}  |  DOWN: {down_count}", 
                   (20, 85), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
        
        # Affichage des commandes
        cv2.putText(frame, "'u' = UP  |  'd' = DOWN  |  'q' = QUIT", 
                   (w - 500, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(frame, f"Session: +{session_up} UP | +{session_down} DOWN", 
                   (w - 500, 85), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (200, 200, 200), 1)
        
        # Affichage des instructions détaillées
        cv2.putText(frame, "Position HAUTE = Bras completement etendus", 
                   (20, h - 80), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 1)
        cv2.putText(frame, "Position BASSE = Bras ploie a 90 degrees", 
                   (20, h - 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 200, 255), 1)
        
        # Afficher
        cv2.imshow("Capture Dataset - Pompes", frame)
        
        # Attendre une commande (délai 1ms pour responsivité)
        key = cv2.waitKey(1) & 0xFF
        
        if key == ord('u'):
            # Enregistrer position UP
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            filename = os.path.join(UP_DIR, f"up_{up_count:04d}_{timestamp}.jpg")
            cv2.imwrite(filename, frame)
            up_count += 1
            session_up += 1
            print(f" Image UP #{up_count:3d} enregistrée (session: +{session_up})")
            
        elif key == ord('d'):
            # Enregistrer position DOWN
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
            filename = os.path.join(DOWN_DIR, f"down_{down_count:04d}_{timestamp}.jpg")
            cv2.imwrite(filename, frame)
            down_count += 1
            session_down += 1
            print(f" Image DOWN #{down_count:3d} enregistrée (session: +{session_down})")
            
        elif key == ord('q'):
            print("\n" + "=" * 60)
            print("  Arrêt de la capture...")
            print("=" * 60)
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    # Affichage du résumé
    total_images = up_count + down_count
    balance = " ÉQUILIBRÉ" if abs(up_count - down_count) <= 2 else "⚠️  DÉSÉQUILIBRÉ"
    
    print(f"\n Résumé de la session:")
    print(f"  ✓ Images UP (position haute): {up_count}")
    print(f"  ✓ Images DOWN (position basse): {down_count}")
    print(f"  ✓ Total: {total_images} images")
    print(f"  ✓ Équilibre: {up_count}/{down_count} {balance}")
    
    if total_images == 0:
        print(f"\n  Aucune image capturée. Réessayez!")
    elif total_images < 50:
        print(f"\n Recommandation: Capturez au minimum 50 images par position (actuellement: {min(up_count, down_count)})")
    else:
        print(f"\n Excellent! Dataset prêt pour l'entraînement.")

if __name__ == "__main__":
    capture_images()
